* project 目錄為 `End-to-End-Machine-Learning-Pipeline`，以下路徑均以此為 根目錄

* `src/mlProject/constants/__init__.py` 下定義了 

	```python
	CONFIG_FILE_PATH 
	PARAMS_FILE_PATH 
	SCHEMA_FILE_PATH
	```

* `src/mlProject/constants/__init__.py` 要先完成 (針對 5. Update the configuration manager )

# 自行實作

In [1]:
!git clone https://github.com/henrykohl/MLOps-Foundation.git

Cloning into 'MLOps-Foundation'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 345 (delta 0), reused 0 (delta 0), pack-reused 330 (from 1)
Receiving objects: 100% (345/345), 3.08 MiB | 32.22 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [ ]:
!conda create -n mlproj python=3.8 -y

In [3]:
%cd MLOps-Foundation/End-to-End-Machine-Learning-Pipeline/

/content/MLOps-Foundation/End-to-End-Machine-Learning-Pipeline


In [4]:
%pwd

'/content/MLOps-Foundation/End-to-End-Machine-Learning-Pipeline'

In [ ]:
!source activate mlproj; pip install -r requirements.txt

* 為了簡化，將 logger 移除
  ```python
  from mlProject import logger
  ```

* colab 操作時，將 `01_data_ingestion.ipynb` 中，自行實作的執行部分複製過來，先執行。

In [8]:
%%bash
source activate mlproj

python

import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

import os

class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-12-13 16:37:02,761: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-13 16:37:02,762: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-13 16:37:02,764: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-13 16:37:02,765: INFO: common: created directory at: artifacts]
[2025-12-13 16:37:02,765: INFO: common: created directory at: artifacts/data_validation]


# Lecture Demo

In [ ]:
import os
os.chdir("../")
%pwd

'd:\\Bappy\\Live Sessions\\Euron\\MLOPs Masters Batch\\End-to-End-Machine-Learning-Pipeline'

## 4. Update the entity
* project 對應 `src/mlProject/entity/config_entity.py`

In [ ]:
import pandas as pd

In [ ]:
## 觀察用，非必要
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path ## 其實是 unzip data file 路徑
    all_schema: dict     ## 之後實例化時，儲存的類型是 ConfigBox

## 5. Update the configuration manager
* project 對應 `src/mlProject/config/configuration.py`

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,   ## 輸出: PosixPath("config/config.yaml")
        params_filepath = PARAMS_FILE_PATH,   ## 輸出: PosixPath("params.yaml")
        schema_filepath = SCHEMA_FILE_PATH):  ## 輸出: PosixPath("schema.yaml")

        self.config = read_yaml(config_filepath) ## 輸出: ConfigBox({...}); config.artifacts_root 是 str
        self.params = read_yaml(params_filepath) ## 輸出: ConfigBox({...})
        self.schema = read_yaml(schema_filepath) ## 輸出: ConfigBox({...})

        create_directories([self.config.artifacts_root]) ## 建立目錄 artifacts; config.artifacts_root 是 str



    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation  ## 輸出: ConfigBox({...})
        schema = self.schema.COLUMNS          ## 輸出: ConfigBox({...})

        create_directories([config.root_dir]) ## 建立目錄 artifacts/data_validation; config.root_dir 是 str

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,                ## config.root_dir 是 str
            STATUS_FILE=config.STATUS_FILE,          ## config.STATUS_FILE 是 str
            unzip_data_dir = config.unzip_data_dir,  ## config.unzip_data_dir 是 str
            all_schema=schema,                       ## schema 是 ConfigBox 
        )

        return data_validation_config

## 6. Update the components
* project 對應 `src/mlProject/components/data_ingestion.py`

In [ ]:
import os
from mlProject import logger

In [ ]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir) ## config.unzip_data_dir 是 str: csv 檔路徑
            all_cols = list(data.columns) ## ['fixed acidity', 'volatile acidity', ..., 'quality']

            all_schema = self.config.all_schema.keys() ## dict_keys(['fixed acidity',...,'quality'])


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e

* 在 class DataValiadtion 中，for 迴圈中的邏輯可能會有問題，如果 validation_status 先 False 再 True，最後只會顯示 True，如果是先 True 在 False，最後只會顯示 False
	```python
	for col in all_cols:
		if col not in all_schema:
			validation_status = False
			with open(self.config.STATUS_FILE, 'w') as f:
				f.write(f"Validation status: {validation_status}")
		else:
			validation_status = True
			with open(self.config.STATUS_FILE, 'w') as f:
				f.write(f"Validation status: {validation_status}") 
	```

## 7. Update the pipeline
* project 對應 `src/mlProject/pipeline/stage_01_data_ingestion.py`

* 完成後，建立 `artifacts/data_validation` 資料夾內包含 'status.txt' 檔案

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-01-12 12:40:28,683: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-12 12:40:28,684: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-12 12:40:28,686: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-12 12:40:28,686: INFO: common: created directory at: artifacts]
[2025-01-12 12:40:28,687: INFO: common: created directory at: artifacts/data_validation]
